In [1]:
from pfs.drp.stella import DetectorMap
import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.afw.detection as afwDetection
from  pfs.utils.dummyCableB import DummyCableBDatabase

# Define drp repo

In [2]:
rerun = "sm1-march2020"
repo = "drp"
drpPath = "/home/fmadec/pfs/subaruData"

SM = "SM1"
site = "Subaru"


# Select a visit and a cam

In [3]:
cam = "r1"

visit = 1376

In [4]:
arm = cam[0]

In [5]:
butler = dafPersist.Butler(f"{drpPath}/{repo}/rerun/{rerun}/detrend/")

In [6]:
calExp = butler.get("calexp", visit=visit, arm=arm)
calexfilePath = butler.getUri("calexp", visit=visit, arm=arm)
print(calexfilePath)

/home/fmadec/pfs/subaruData/drp/rerun/sm1-march2020/detrend/calExp/2020-04-07/v001376/calExp-SA001376r1.fits


# Retrieve some informations on this visit

## Lamp used

In [7]:
rbutler = dafPersist.Butler(f"{drpPath}/{repo}/")
rbutler.getKeys('raw')
lamps = rbutler.queryMetadata('raw', ['lamps'], visit=visit, arm=arm) 
print(f"Lamp used: {lamps}")

Lamp used: ['HgAr']


## PFS Config Id (so fibers used)

In [8]:
pfsConfig = butler.get("pfsConfig", visit=visit)
pfsConfig.pfsDesignId
dcb = DummyCableBDatabase()
fiberSetup = dcb.interpret(pfsConfig.pfsDesignId)
print(f"DCB fibers config: {fiberSetup}")
fiberIds = dcb.getFiberIds(*fiberSetup)
print(f"Fiber used: {fiberIds}")

DCB fibers config: ['green', 'red1', 'red4', 'red8']
Fiber used: [  2  63 192 255 339 401 464 525 587 650]


# Display image in DS9

## display image

In [9]:
display = afwDisplay.Display(1)

In [10]:
# display image
display.setMaskTransparency(70)
display.mtv(calExp, title=f"{visit}")
display.scale("linear", "zscale")

## display variance

In [11]:
display = afwDisplay.Display(2)

In [12]:
# display variance
display.setMaskTransparency(70)
display.mtv(calExp.variance, title=f"Variance {visit}")
display.scale("linear", "zscale")

## Change mask 

### Interpreting displayed mask colors


In [13]:
mask = calExp.getMask()
for maskName, maskBit in mask.getMaskPlaneDict().items():
    print('{}: {}'.format(maskName, display.getMaskPlaneColor(maskName)))

BAD: red
BAD_FLAT: None
CR: magenta
DETECTED: blue
DETECTED_NEGATIVE: cyan
EDGE: yellow
FIBERTRACE: None
INTRP: green
NO_DATA: orange
REJECTED_LINES: None
SAT: green
SUSPECT: yellow
UNMASKEDNAN: None


### disable all mask

In [14]:
mask = calExp.getMask()
for maskName, maskBit in mask.getMaskPlaneDict().items():
    display.setMaskPlaneColor(maskName, "IGNORE")
    print('{}: {}'.format(maskName, display.getMaskPlaneColor(maskName)))

BAD: IGNORE
BAD_FLAT: IGNORE
CR: IGNORE
DETECTED: IGNORE
DETECTED_NEGATIVE: IGNORE
EDGE: IGNORE
FIBERTRACE: IGNORE
INTRP: IGNORE
NO_DATA: IGNORE
REJECTED_LINES: IGNORE
SAT: IGNORE
SUSPECT: IGNORE
UNMASKEDNAN: IGNORE


### specify some maskPlane

In [15]:
maskPlane = "CR"
calExp.getMaskedImage().getMask().addMaskPlane(maskPlane)
display.setMaskPlaneColor(maskPlane, "magenta")
maskPlane = "SAT"
calExp.getMaskedImage().getMask().addMaskPlane(maskPlane)
display.setMaskPlaneColor(maskPlane, "green")

### have to redo display to apply mask change

In [16]:
display = afwDisplay.Display(1)
# display image
display.setMaskTransparency(70)
display.mtv(calExp, title=f"{visit}")
display.scale("linear", "zscale")